In [ ]:
from pprint import pprint
from demo.server.demo_helpers import get_schema, execute_query, pretty_print_data

## The autogenerated schema is real GraphQL SDL

In [ ]:
print(get_schema())

## Queries are in real GraphQL syntax, but have different semantics

### Naming fields does not cause them to be output
- Explicit `@output` directive required to request that a field is output.
- This allows filtering on fields without outputting them, and many other nice things.

In [ ]:
# Get some details about the BOS airport.
query = '''{
    Airport {
        name @output(out_name: "airport_name")
        city_served @output(out_name: "airport_city")
        iata_code @filter(op_name: "=", value: ["$code"])  # filtered but not output
    }
}'''
args = {
    'code': 'BOS',
}
_, result = execute_query(query, args)
pretty_print_data(result)

### Why not just use `input`-typed objects for filtering? Several reasons:
- We already discussed that the necessary autogenerated `input` objects are going to be massive. On a multi-million-line schema (not counting documentation), this adds up quickly!
- Besides, what if we want to apply a particular kind of filter more than once on a given field?

In [ ]:
# Get the airlines and their registration countries where
# the airline name contains both "Airways" and "Jet".
query = '''{
    Airline {
        name @filter(op_name: "has_substring", value: ["$name_substring1"])
             @filter(op_name: "has_substring", value: ["$name_substring2"])
             @output(out_name: "airline")
                
        out_Airline_RegisteredIn {
            name @output(out_name: "country")
        }
    }
}'''
args = {
    'name_substring1': 'Airways',
    'name_substring2': 'Jet',
}
_, result = execute_query(query, args)
pretty_print_data(result)

### The output format is shaped like a dataframe (list of dicts, rows and columns)
- This is what the database returned as well; producing "proper" nested GraphQL requires postprocessing.
- We avoid doing any kind of postprocessing in general since that costs performance.

In [ ]:
pprint(result)

### We also expose advanced database features via directives, such as recursive JOINs (edge traversals)
- Check out the compiler's `@recurse`, `@optional`, and `@tag` directives: https://github.com/kensho-technologies/graphql-compiler

In [ ]:
# Get the names of all countries in Europe that end in "land".
query = '''{
    Region {
        name @filter(op_name: "=", value: ["$region"])
        
        out_GeographicArea_SubArea @recurse(depth: 5) {
            ... on Country {
                name @output(out_name: "country_name") @filter(op_name: "ends_with", value: ["$suffix"])
            }
        }
    }
}'''
args = {
    'region': 'Europe',
    'suffix': 'land',
}
_, result = execute_query(query, args)
pretty_print_data(result)